<a href="https://colab.research.google.com/github/vatsDivyank/Face-Recoginition-TinyML/blob/main/Divyank_not_Divyank_FineTuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf

In [2]:
!cp "/content/drive/MyDrive/Preprocessing_images.py" "/content"

In [ ]:
from Preprocessing_images import DivyankNotDivyank


In [ ]:
MODELS_DIR = os.path.abspath("/content/drive/MyDrive/MODEL_DIR")
MODEL_FILE_PATH_PB= os.path.join(MODELS_DIR, "divyank_not_divyank_FineTuned_Mobilenet.pb")
MODEL_FILE_PATH_h5= os.path.join(MODELS_DIR, "divyank_not_divyank_FineTuned_Mobilenet.h5")
MODEL_TFLITE = os.path.join(MODELS_DIR, 'divyank_not_divyank_FineTuned_Mobilenet.tflite')
MODEL_TFLITE_MICRO = os.path.join(MODELS_DIR , 'divyank_not_divyank_Mobilenet.cc')

if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)

LOGS_DIR = os.path.abspath("/content/drive/MyDrive/Logs_Dir/Logs_25.02")
if not os.path.exists(LOGS_DIR):
    os.mkdir(LOGS_DIR)

In [ ]:
IMAGENET_SIZE = 96
IMAGENET_DEPTH = 3
IMAGENET_SHAPE = (IMAGENET_SIZE, IMAGENET_SIZE, IMAGENET_DEPTH)

def build_model(img_shape, num_classes) -> Model:
    base_model = MobileNetV2(
        alpha = 1.0,
        dropout = 0.05
        include_top=False,
        weights="imagenet",
        input_shape=IMAGENET_SHAPE
    )

    num_layers = len(base_model.layers)
    print(f"Number of layers in the base model: {num_layers}")
    fine_tune_at = num_layers - 10
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    input_img = Input(shape=img_shape)
    x = Rescaling(scale=2.0, offset=-1.0)(input_img)
    x = Resizing(height=IMAGENET_SIZE, width=IMAGENET_SIZE)(x)
    x = Concatenate()([x, x, x])
    x = base_model(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=num_classes)(x)
    y_pred = Activation("softmax")(x)

    model = Model(
        inputs=[input_img],
        outputs=[y_pred]
    )

    model.summary()

    return model


In [ ]:
%load_ext tensorboard
%tensorboard --logdir {train_logs} 

In [ ]:
if __name__ == "__main__":

    data = DivyankNotDivyank()
    data.data_augmentation(augment_size= 1000)
    x_train_, x_val_,y_train_, y_val_ = data.get_splitted_train_validation_set()
    x_test , y_test = data.get_test_set()

    img_shape = data.img_shape
    num_classes = data.num_classes

    # Global params
    epochs = 10
    batch_size = 100


    model = build_model(
        img_shape,
        num_classes,
    )
    opt = Adam(learning_rate=7e-4)

    model.compile(
        loss="categorical_crossentropy",
        optimizer=opt,
        metrics=["accuracy"]
    )
    
    
    
    tb_callback = TensorBoard(
        log_dir=LOGS_DIR,
        histogram_freq=0,
        profile_batch=0
    )
  


    model.fit(
        x=x_train_ ,
        y= y_train_,
        verbose=1,
        epochs=epochs,
        #callbacks=[tb_callback],
        validation_data=(x_val_, y_val_),
    )
    model.save(MODEL_FILE_PATH_PB)
    model.save(MODEL_FILE_PATH_h5)

2023424/2019640 [==============================] - 0s 0us/step
Number of layers in the base model: 154
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 96, 96, 1)]  0                                            
__________________________________________________________________________________________________
rescaling_5 (Rescaling)         (None, 96, 96, 1)    0           input_14[0][0]                   
__________________________________________________________________________________________________
resizing_5 (Resizing)           (None, 96, 96, 1)    0           rescaling_5[0][0]                
__________________________________________________________________________________________________
concatenate_5 (Concatenate)     (None, 96, 96, 3)    0           resizing_5[0][0]       

INFO:tensorflow:Assets written to: /content/drive/MyDrive/MODEL_DIR/divyank_not_divyank_FineTuned_Mobilenet.pb/assets


In [ ]:
basic_model_size_h5 = os.path.getsize(MODEL_FILE_PATH_h5)/1024
print("model size_h5: {} KB" .format(basic_model_size_h5))

#model size_h5: 14901.796875 KB
# model size_h5: 25231.375 KB Mobile net V1 , aplha = 1.0

model size_h5: 3430.203125 KB


In [ ]:

#Converting the Model to Tensorflow Lite format with float16 quantization
converter_float16 = tf.lite.TFLiteConverter.from_saved_model(MODEL_FILE_PATH_PB)
converter_float16.optimizations = [tf.lite.Optimize.DEFAULT]
converter_float16.target_spec.supported_types = [tf.float16]
tflite_model_float16 = converter_float16.convert()

#Save to disk
model_float16 = (open("model_quant16.tflite","wb").write(tflite_model_float16))/1024
print("float16 Model size %d KB" % model_float16)

#Converting the Model to the Tensorflow Lite format with dynamic range Quantization
converter_dynamic = tf.lite.TFLiteConverter.from_saved_model(MODEL_FILE_PATH_PB)
converter_dynamic.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_dynamic =  converter_dynamic.convert()

#Save to Disk
model_dynamic_size = (open("model_dynamic.tflite","wb").write(tflite_model_dynamic))/1024
print("Dynamic Model size %d KB" % model_dynamic_size)


#Converting the Model into Tensorflow Lite format with full Interger quantization

#This Quantization requires creation of representative Dataset
images = tf.cast(x_test, tf.float32)
test_data = tf.data.Dataset.from_tensor_slices(images).batch(1)
def representative_dataset_gen():
  for input in test_data.take(100):
    yield[input]

converter_full_integer = tf.lite.TFLiteConverter.from_saved_model(MODEL_FILE_PATH_PB)
converter_full_integer.optimizations = [tf.lite.Optimize.DEFAULT]
converter_full_integer.representative_dataset = representative_dataset_gen
converter_full_integer.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_full_integer.inference_input_type = tf.int8
converter_full_integer.inference_output_type = tf.float32
tflite_model_full_integer = converter_full_integer.convert()

# Save to disk
model_full_integer=(open("model_full_integer.tflite", "wb").write(tflite_model_full_integer))/ 1024
print("Full Integer Model Size %d KB" % model_full_integer)



float16 Model size 798 KB
Dynamic Model size 573 KB
Full Integer Model Size 651 KB


In [ ]:
# Install xxd if it is not available
!apt-get update && apt-get -qq install xxd
# Convert to a C source file
!xxd -i "model_full_integer.tflite" > {MODEL_TFLITE_MICRO}

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


In [ ]:
!cat {MODEL_TFLITE_MICRO}

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  0x93, 0x11, 0x26, 0x3d, 0xee, 0xf8, 0xee, 0x02, 0x04, 0xe5, 0x24, 0x34,
  0xf8, 0xc9, 0x1f, 0xe1, 0x05, 0x11, 0x0f, 0x1b, 0x3f, 0xf6, 0xd5, 0xfc,
  0x0d, 0x24, 0x16, 0xb6, 0x1b, 0xe1, 0x1e, 0x5f, 0xcb, 0xd3, 0x60, 0x24,
  0x1b, 0x0a, 0xa2, 0xe4, 0xe0, 0xf7, 0x33, 0xbe, 0x1c, 0x26, 0x27, 0xee,
  0x6a, 0x28, 0x05, 0x05, 0xef, 0xc6, 0x5c, 0x25, 0xdd, 0x17, 0x50, 0x18,
  0x25, 0x10, 0x04, 0x1a, 0xeb, 0xfd, 0xf4, 0xdb, 0xef, 0x49, 0xc4, 0xed,
  0xe5, 0x06, 0x02, 0xf2, 0xf3, 0xee, 0x21, 0xfe, 0xe4, 0xc7, 0xe1, 0xe5,
  0x25, 0xe6, 0x08, 0x5f, 0x01, 0xe5, 0xef, 0x03, 0xa1, 0xd8, 0x07, 0xee,
  0x08, 0x0c, 0xc1, 0x23, 0x03, 0x06, 0xd3, 0x27, 0xe0, 0x1c, 0xf9, 0xf7,
  0x11, 0x11, 0x25, 0xfa, 0xca, 0xbf, 0x0d, 0xf2, 0x1c, 0x17, 0xab, 0xdf,
  0x2c, 0xd6, 0xe0, 0x29, 0x0c, 0x19, 0xff, 0xe5, 0xe8, 0x50, 0xd7, 0xb9,
  0xd5, 0xe9, 0x51, 0x0a, 0x24, 0xdb, 0xfe, 0xfc, 0xf3, 0xf8, 0xea, 0x28,
  0x0b, 0xfc, 0x07, 0xe9, 0x49, 0xfb, 0xf5, 0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
  model.load_weights (MODEL_FILE_PATH)
  TEST_PHOTO_DIR = os.path.abspath("/content/drive/MyDrive/Folder_to_test_photos")
  image_names = [f for f in os.listdir(TEST_PHOTO_DIR) if ".jpg"in f or".jpeg" in f or ".png" in f]
  print(image_names)
  class_names = ["Divyank", "not_Divyank"]
  for image_file_name in image_names:
        image_file_path = os.path.join(TEST_PHOTO_DIR, image_file_name)
        print(image_file_path)
        x = data.load_and_preprocess_custom_image(image_file_path)
        x = np.expand_dims(x, axis=0)
        y_pred = model.predict(x)[0]
        y_pred_class_idx = np.argmax(y_pred)
        y_pred_prob = y_pred[ y_pred_class_idx]
        y_pred_class_name=class_names[y_pred_class_idx]
        plt.imshow(x.reshape(96, 96, 1))
        plt.title(f"Pred class:{y_pred_class_name}, Prob:{y_pred_prob}")
        plt.show()